# 전처리된 데이터 호출

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import time

In [3]:
# data_3 에서 판매량이 음수인 경우 제거
df = pd.read_pickle('../Team_data/data_4.pkl')
df

month_id  shop_id  item_id  cnt_month  city_id  cat_id  main_cat  \
827196          3       25     8092        3.0       13      40        11   
827197          3       25     7850        3.0       13      30         8   
827198          3       25     8051        3.0       13      66        14   
827199          3       25     8088        1.0       13      55        13   
827200          3       25     8089        1.0       13      55        13   
...           ...      ...      ...        ...      ...     ...       ...   
8856302        34       45    18454        NaN       20      55        13   
8856303        34       45    16188        NaN       20      64        14   
8856304        34       45    15757        NaN       20      55        13   
8856305        34       45    19648        NaN       20      40        11   
8856306        34       45      969        NaN       20      37        11   

         sub_cat  year  month  ...  price_mean_mean_cnt_3  \
827196         6  2013      4  ...                    0.0   
827197        54  2013      4  ...                    0.0   
827198        47  2013      4  ...                    0.0   
827199         4  2013      4  ...                    0.0   
827200         4  2013      4  ...                    0.0   
...          ...   ...    ...  ...                    ...   
8856302        4  2015     11  ...                    0.0   
8856303       42  2015     11  ...                    0.0   
8856304        4  2015     11  ...                    0.0   
8856305        6  2015     11  ...                    0.0   
8856306        1  2015     11  ...                    0.0   

         item_id/city_id_mean_cnt_1  item_id/city_id_mean_cnt_2  \
827196                     1.000000                    2.142857   
827197                     3.714286                    2.285714   
827198                     0.000000                    0.428571   
827199                     0.285714                    0.000000   
827200                     0.571429                    0.000000   
...                             ...                         ...   
8856302                    0.500000                    0.000000   
8856303                    0.000000                    0.000000   
8856304                    0.000000                    0.000000   
8856305                    0.000000                    0.000000   
8856306                    0.500000                    0.000000   

         item_id/city_id_mean_cnt_3  shop_id/cat_id_mean_cnt_1  \
827196                          0.0                   1.087262   
827197                          0.0                   3.654494   
827198                          0.0                   1.827586   
827199                          0.0                   0.599845   
827200                          0.0                   0.599845   
...                             ...                        ...   
8856302                         0.0                   0.126866   
8856303                         0.0                   0.094488   
8856304                         0.0                   0.126866   
8856305                         0.0                   0.083756   
8856306                         0.0                   0.128205   

         shop_id/cat_id_mean_cnt_2  shop_id/cat_id_mean_cnt_3  hd_mean_1  \
827196                    1.049296                   0.000000        1.0   
827197                    2.279188                   0.000000        0.5   
827198                    1.389474                   0.000000        0.0   
827199                    0.562107                   0.000000        1.0   
827200                    0.562107                   0.000000        0.0   
...                            ...                        ...        ...   
8856302                   0.106061                   0.128242        1.0   
8856303                   0.000000                   0.000000        0.0   
8856304                   0.106061                   0.128242        

In [4]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

In [5]:
df.columns

Index(['month_id', 'shop_id', 'item_id', 'cnt_month', 'city_id', 'cat_id',
       'main_cat', 'sub_cat', 'year', 'month', 'sales_month', 'cnt_month_1',
       'cnt_month_2', 'cnt_month_3', 'item_id_mean_cnt_1',
       'item_id_mean_cnt_2', 'item_id_mean_cnt_3', 'sub_cat_mean_cnt_1',
       'sub_cat_mean_cnt_2', 'sub_cat_mean_cnt_3', 'price_mean_mean_cnt_1',
       'price_mean_mean_cnt_2', 'price_mean_mean_cnt_3',
       'item_id/city_id_mean_cnt_1', 'item_id/city_id_mean_cnt_2',
       'item_id/city_id_mean_cnt_3', 'shop_id/cat_id_mean_cnt_1',
       'shop_id/cat_id_mean_cnt_2', 'shop_id/cat_id_mean_cnt_3', 'hd_mean_1',
       'hd_mean_2', 'hd_mean_3'],
      dtype='object')

In [6]:
df[df['month_id']<34]['cnt_month']

827196     3.0
827197     3.0
827198     3.0
827199     1.0
827200     1.0
          ... 
8642102    0.0
8642103    0.0
8642104    0.0
8642105    0.0
8642106    0.0
Name: cnt_month, Length: 7814911, dtype: float32

In [7]:
x_train = df[df['month_id']<33].drop('cnt_month', axis = 1)
y_train = df[df['month_id']<33]['cnt_month']
y_train = y_train.clip(0,20)

x_test = df[df['month_id']==33].drop('cnt_month', axis = 1)
y_test = df[df['month_id']==33]['cnt_month']
y_test = y_test.clip(0,20)

## 주성분 분석 시행
- n 값은 임의로 설정, n = 10
- x_train, x_test, submit_test 값에 각각 pca(n=10) 시행

In [8]:
from sklearn.decomposition import PCA

In [9]:
pca = PCA(n_components=10)
x_train = pca.fit_transform(x_train)
x_test = pca.fit_transform(x_test)

---

In [10]:
# 가장 우수한 성능을 보인 모델 선택
# max_depth = 9, min_samples_leaf = 5
time_start = time.time()

tree = DecisionTreeRegressor(max_depth = 9, min_samples_leaf = 5,
                            random_state=42)
tree.fit(x_train, y_train)
y_pred = tree.predict(x_test)

rmse = mean_squared_error(y_test, y_pred)**0.5

time_finish = time.time() - time_start

print('time :{}, rmse = {}'.format(time_finish, rmse))

time :136.12790369987488, rmse = 0.9879209599360188


csv 파일 제출

In [11]:
# 제출 파일 생성 - 1
submit_test = df[df['month_id']==34].drop('cnt_month', axis = 1)
submit_test = pca.fit_transform(submit_test)

In [12]:
# 제출 파일 생성 - 2
submit_pred = tree.predict(submit_test)

In [24]:
df_test = pd.read_csv('../test.csv')
submission = pd.DataFrame({"ID": df_test.index, "item_cnt_month": submit_pred})
submission.to_csv('./DecisionTreeRegressor/DTR_submission_06.csv', index=False)

---